In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
import gc
from tensorflow.python.keras import layers, Sequential,losses, metrics, optimizers, callbacks, models
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.applications import vgg16
from tensorflow.keras.optimizers import SGD, Adam

In [2]:
image_height = 48
image_width = 48
emotions_count = 8
emotion_labels = ['neutral', 'happiness', 'surprise', 'sadness', 'anger', 'disgust', 'fear', 'contempt']

In [3]:
image_path = "./dataset_f+r20/images.npy"
emotion_path = "./dataset_f+r20/emotions_multi.npy"

images = np.load(image_path)
images = tf.convert_to_tensor(images)
images = layers.Rescaling(1./127.5, offset=-1)(images)
images = tf.image.grayscale_to_rgb(images)

emotions = np.load(emotion_path)
emotions = tf.convert_to_tensor(emotions)
    
training_samples = 28317 * 4
validation_samples = 3541 * 4
training_size = training_samples + validation_samples
    
training_images = images[:training_size]
test_images = images[training_size:]
training_emotions = emotions[:training_size]
test_emotions = emotions[training_size:]

In [4]:
tf.config.run_functions_eagerly(True)
def model_acc(y_true, y_pred):
    size = y_true.shape[0]
    acc = 0
    for i in range(size):
        true = y_true[i]
        pred = y_pred[i]           
        index_max = tf.argmax(pred).numpy()
        if true[index_max].numpy()==tf.reduce_max(true).numpy():
            acc += 1
    return acc/size

In [9]:
def train_SGD(model, learning_rate, loss, num_epochs, batch_size):
    for i in range(4):
        model.compile(optimizer=SGD(learning_rate=0.1**(i+1), momentum=0.9), 
                      loss=loss, 
                      metrics = [model_acc])
        history = model.fit(x=training_images,
                            y=training_emotions,
                            batch_size=batch_size,
                            epochs=10,
                            validation_data=(test_images, test_emotions))
    del model
    gc.collect()
    return history

def train_KLD(model, learning_rate, loss, num_epochs, batch_size):
    model.compile(optimizer=Adam(learning_rate=learning_rate), 
                    loss=loss, 
                    metrics = [model_acc])
    history = model.fit(x=training_images,
                        y=training_emotions,
                        batch_size=batch_size,
                        epochs=num_epochs,
                        validation_data=(test_images, test_emotions))
    del model
    gc.collect()
    return history

In [6]:
def create_model(base_model):
    base_model.trainable=True
    return Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(4096, activation='relu'),
        layers.Dense(4096, activation='relu'),
        layers.Dense(emotions_count, activation='softmax'),
    ])

In [11]:
if not os.path.isdir('./results/'):
    os.mkdir('./results/')

learning_rate = 1e-4
num_epochs = 40
batch_size = 32
loss = losses.MeanSquaredError()

base_model = vgg16.VGG16(include_top=False, weights='imagenet', input_shape=(48,48,3))
history_save_path = './history/SGD_self_adj_lr.txt'
model = create_model(base_model)
history = train_SGD(model, learning_rate, loss, num_epochs, batch_size)
with open(history_save_path, 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

Epoch 1/10
3983/3983 [==============================] - 276s 69ms/step - loss: 0.0239 - model_acc: 0.7324 - val_loss: 0.0185 - val_model_acc: 0.7641
Epoch 2/10
3983/3983 [==============================] - 278s 70ms/step - loss: 0.0146 - model_acc: 0.8190 - val_loss: 0.0145 - val_model_acc: 0.8176
Epoch 3/10
3983/3983 [==============================] - 281s 70ms/step - loss: 0.0115 - model_acc: 0.8526 - val_loss: 0.0131 - val_model_acc: 0.8264
Epoch 4/10
3983/3983 [==============================] - 278s 70ms/step - loss: 0.0093 - model_acc: 0.8785 - val_loss: 0.0125 - val_model_acc: 0.8380
Epoch 5/10
3983/3983 [==============================] - 279s 70ms/step - loss: 0.0076 - model_acc: 0.8999 - val_loss: 0.0119 - val_model_acc: 0.8436
Epoch 6/10
3983/3983 [==============================] - 280s 70ms/step - loss: 0.0062 - model_acc: 0.9185 - val_loss: 0.0119 - val_model_acc: 0.8478
Epoch 7/10
3983/3983 [==============================] - 269s 68ms/step - loss: 0.0051 - model_acc: 0.9335 

In [12]:
loss = losses.KLDivergence()
base_model = vgg16.VGG16(include_top=False, weights='imagenet', input_shape=(48,48,3))
history_save_path = './history/KLD.txt'
model = create_model(base_model)
history = train_KLD(model, learning_rate, loss, num_epochs, batch_size)
with open(history_save_path, 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

Epoch 1/40
3983/3983 [==============================] - 282s 71ms/step - loss: 0.5109 - model_acc: 0.7410 - val_loss: 0.3755 - val_model_acc: 0.8041
Epoch 2/40
3983/3983 [==============================] - 282s 71ms/step - loss: 0.3347 - model_acc: 0.8328 - val_loss: 0.3559 - val_model_acc: 0.8173
Epoch 3/40
3983/3983 [==============================] - 282s 71ms/step - loss: 0.2715 - model_acc: 0.8678 - val_loss: 0.3207 - val_model_acc: 0.8441
Epoch 4/40
3983/3983 [==============================] - 282s 71ms/step - loss: 0.2276 - model_acc: 0.8924 - val_loss: 0.3046 - val_model_acc: 0.8490
Epoch 5/40
3983/3983 [==============================] - 282s 71ms/step - loss: 0.1937 - model_acc: 0.9103 - val_loss: 0.3072 - val_model_acc: 0.8493
Epoch 6/40
3983/3983 [==============================] - 282s 71ms/step - loss: 0.1647 - model_acc: 0.9240 - val_loss: 0.3136 - val_model_acc: 0.8456
Epoch 7/40
3983/3983 [==============================] - 281s 71ms/step - loss: 0.1388 - model_acc: 0.9330 